# Week 12 Preppin Data Challenge
## Chin & Beard Suds Co
**Background**
Would you believe that Chin & Beard Suds Co have encountered yet more messy data? It seems someone was trying to be helpful by creating an aggregated view of sales per week for each scent of soap. However, in doing so we've lost the lower level of detail of the product sizes that make up these sales for each scent. We really need this for other analysis we've been carrying out!

Fortunately, we know what percentage of sales each product size makes up for each product in each week. Unfortunately, the data isn't stored in a way that will make it easy to join all the necessary information together.

**Requirements**
* Input data
* Our final output requires the Date to be in in the Year Week Number format. 
* We don't care about any product sizes that make up 0% of sales.
* In the Lookup Table, it seems the Product ID and Size have been erroneously concatenated. These need to be separated.  
* You'll need to do some cleaning of the Scent fields to join together the Total Sales and the Lookup Table.
* Calculate the sales per week for each scent and product size.
* Output the data

**Output**
5 Data Fields:
* Year Week Number
* Scent
* Product Type
* Size
* Sales
* 307 rows (308 including headers)

In [1]:
# import packages
import pandas as pd

In [8]:
# load data
dfTotSales = pd.read_excel('PD week 12 input.xlsx', sheet_name = 'Total Sales')
dfPctSales = pd.read_excel('PD week 12 input.xlsx', sheet_name = 'Percentage of Sales')
dfLookup = pd.read_excel('PD week 12 input.xlsx', sheet_name = 'Lookup Table')